In [7]:
import json
from datetime import datetime , timezone
import time
import numpy as np
import codecs
import struct
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import pycorrelate
import collections
import itertools
from mpl_toolkits import mplot3d
import cv2
import imutils
import os
from scipy.spatial.transform import Rotation as R

pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_rows', 1000)
%matplotlib inline

### General Stats
hours of video recorded and glasses data collected:

In [8]:
metadata = pickle.load(open('./cleaned_data/metadata.p', 'rb'))
total_video_min, total_glasses_min = 0, 0
for k in metadata:
    total_glasses_min += sum(metadata[k]['glasses_session_durations_min'])
    total_video_min += metadata[k]['vid_duration_sec']/60
    
print('Total Video Recorded: %d:%d hr:min'%(int(total_video_min/60), total_video_min%60))
print('Total Glasses Recorded: %d:%d hr:min'%(int(total_glasses_min/60),total_glasses_min%60))
print('Roughly',45/5,'hrs per participant')

Total Video Recorded: 45:13 hr:min
Total Glasses Recorded: 47:2 hr:min
Roughly 9.0 hrs per participant


### Blinking Stats
hand annotations, how fast people blink, etc

In [15]:
sessions = ['juliana_1', 'beata_1', 'irmandy_2', 'david_3b', 'patrick_3']

In [10]:
t_total_blinks, t_total_dur_sec, t_total_num_skipped, t_total_dur_skipped, t_of_blinks_skipped = 0, 0, 0, 0, 0

for SESSION in sessions:
    hand_annotations = pickle.load(open('./cleaned_data/' + SESSION + '/hand_annotations.p', "rb" ))
    vid_meta_data    = pickle.load(open('./cleaned_data/' + SESSION + '/video_blink_sessions.p', "rb" ))
    
    total_blinks, total_dur_sec, total_num_skipped, total_dur_skipped, of_blinks_skipped = 0, 0, 0, 0, 0
    for k in hand_annotations:
        try:
            total_blinks += len(hand_annotations[k])
            total_dur_sec += vid_meta_data['duration_sessions_sec'][k]
        except:
            total_num_skipped +=1
            total_dur_skipped += vid_meta_data['duration_sessions_sec'][k]
            of_blinks_skipped += vid_meta_data['blinks_per_session'][k]

    print(SESSION,'%d hand-annotated blinks with %3.2f secs of recording. %2.2f blinks/min.'%(total_blinks, total_dur_sec, 60*total_blinks/total_dur_sec))
    print(SESSION,'skipped %d snippets with %3.2f sec of recording, including %d blinks found by openface.'%(total_num_skipped, total_dur_skipped, of_blinks_skipped))
    print('')
    t_total_blinks      += total_blinks
    t_total_dur_sec     += total_dur_sec
    t_total_num_skipped += total_num_skipped
    t_total_dur_skipped += total_dur_skipped
    t_of_blinks_skipped += of_blinks_skipped
    
print('-'*30)
print(' OVERALL ')
print('%d hand-annotated blinks with %3.2f secs of recording. %2.2f blinks/min.'%(t_total_blinks, t_total_dur_sec, 60*t_total_blinks/t_total_dur_sec))
print('skipped %d snippets with %3.2f sec of recording, including %d blinks found by openface.'%(t_total_num_skipped, t_total_dur_skipped, t_of_blinks_skipped))
    
print('')

print('over 2 hours of hand-annotated blink footage, skipped about 15 min of footage that openface is confident about.' )
print('captured 2586 blinks, with an overall average blink rate of 21 blinks/min.  Average blink rates per user varied between 11 and 62 blinks/min.' )

juliana_1 516 hand-annotated blinks with 498.80 secs of recording. 62.07 blinks/min.
juliana_1 skipped 3 snippets with 263.56 sec of recording, including 26 blinks found by openface.

beata_1 501 hand-annotated blinks with 2663.91 secs of recording. 11.28 blinks/min.
beata_1 skipped 4 snippets with 542.39 sec of recording, including 28 blinks found by openface.

irmandy_2 518 hand-annotated blinks with 683.97 secs of recording. 45.44 blinks/min.
irmandy_2 skipped 1 snippets with 55.92 sec of recording, including 0 blinks found by openface.

david_3b 507 hand-annotated blinks with 1855.35 secs of recording. 16.40 blinks/min.
david_3b skipped 5 snippets with 296.51 sec of recording, including 80 blinks found by openface.

patrick_3 544 hand-annotated blinks with 1657.78 secs of recording. 19.69 blinks/min.
patrick_3 skipped 8 snippets with 26.11 sec of recording, including 21 blinks found by openface.

------------------------------
 OVERALL 
2586 hand-annotated blinks with 7359.80 secs 

### Results
openface results, our results

In [14]:
# 'RAW'
of_true_pos, g_true_pos, gf_true_pos = 0, 0, 0
of_false_pos, g_false_pos, gf_false_pos = 0, 0, 0
real_pos = 0

for SESSION in sessions:
    results = pickle.load(open('./cleaned_data/' + SESSION + '/final_results_handtuned.p', "rb" ))
    print(SESSION)
    print(SESSION[:4],'openface    : Sensitivity/False Disc: %3.2f , %3.2f'%(results['openface']['sensitivity'], results['openface']['false_discovery']))
    print(SESSION[:4],'glasses     : Sensitivity/False Disc: %3.2f , %3.2f'%(results['glasses']['sensitivity'], results['glasses']['false_discovery']))
    print(SESSION[:4],'filt glasses: Sensitivity/False Disc: %3.2f , %3.2f'%(results['glasses_filtered']['sensitivity'], results['glasses_filtered']['false_discovery']))
    
    of_true_pos  += results['openface']['true_pos']
    g_true_pos   += results['glasses']['true_pos']
    gf_true_pos  += results['glasses_filtered']['true_pos']
    of_false_pos += results['openface']['false_pos']
    g_false_pos  += results['glasses']['false_pos']
    gf_false_pos += results['glasses_filtered']['false_pos']
    real_pos     += results['openface']['real_pos']
    
print('-'*30)
print(' OVERALL ')
print(' OPENFACE SENS/FALSE DISC %3.2f , %3.2f'%(of_true_pos/real_pos*100, of_false_pos/(of_false_pos+of_true_pos)*100))
print(' GLASS    SENS/FALSE DISC %3.2f , %3.2f'%(g_true_pos/real_pos*100, g_false_pos/(g_false_pos+of_true_pos)*100))
print(' GLASSFIL SENS/FALSE DISC %3.2f , %3.2f'%(gf_true_pos/real_pos*100, gf_false_pos/(gf_false_pos+gf_true_pos)*100))


beata_1
beat openface    : Sensitivity/False Disc: 53.30 , 58.01
beat glasses     : Sensitivity/False Disc: 72.25 , 25.71
beat filt glasses: Sensitivity/False Disc: 69.51 , 24.70
irmandy_2
irma openface    : Sensitivity/False Disc: 66.81 , 20.10
irma glasses     : Sensitivity/False Disc: 84.91 , 27.71
irma filt glasses: Sensitivity/False Disc: 43.53 , 10.62
david_3b
davi openface    : Sensitivity/False Disc: 75.94 , 37.90
davi glasses     : Sensitivity/False Disc: 85.21 , 20.15
davi filt glasses: Sensitivity/False Disc: 85.21 , 20.15
patrick_3
patr openface    : Sensitivity/False Disc: 66.91 , 45.99
patr glasses     : Sensitivity/False Disc: 85.11 , 19.90
patr filt glasses: Sensitivity/False Disc: 83.27 , 16.42
------------------------------
 OVERALL 
 OPENFACE SENS/FALSE DISC 66.68 , 41.56
 GLASS    SENS/FALSE DISC 82.60 , 27.11
 GLASSFIL SENS/FALSE DISC 71.31 , 18.54


In [ ]:
Glasses  80.0/48.1
Openface 53.8/43.9

In [5]:
# 'ALGO'
of_true_pos, g_true_pos, gf_true_pos = 0, 0, 0
of_false_pos, g_false_pos, gf_false_pos = 0, 0, 0
real_pos = 0

for SESSION in sessions:
    results = pickle.load(open('./cleaned_data/' + SESSION + '/final_results_algo.p', "rb" ))
    print(SESSION)
    print(SESSION[:4],'openface    : Sensitivity/False Disc: %3.2f , %3.2f'%(results['openface']['sensitivity'], results['openface']['false_discovery']))
    print(SESSION[:4],'glasses     : Sensitivity/False Disc: %3.2f , %3.2f'%(results['glasses']['sensitivity'], results['glasses']['false_discovery']))
    print(SESSION[:4],'filt glasses: Sensitivity/False Disc: %3.2f , %3.2f'%(results['glasses_filtered']['sensitivity'], results['glasses_filtered']['false_discovery']))
    
    of_true_pos  += results['openface']['true_pos']
    g_true_pos   += results['glasses']['true_pos']
    gf_true_pos  += results['glasses_filtered']['true_pos']
    of_false_pos += results['openface']['false_pos']
    g_false_pos  += results['glasses']['false_pos']
    gf_false_pos += results['glasses_filtered']['false_pos']
    real_pos     += results['openface']['real_pos']
    
print('-'*30)
print(' OVERALL ')
print(' OPENFACE SENS/FALSE DISC %3.2f , %3.2f'%(of_true_pos/real_pos*100, of_false_pos/(of_false_pos+of_true_pos)*100))
print(' GLASS    SENS/FALSE DISC %3.2f , %3.2f'%(g_true_pos/real_pos*100, g_false_pos/(g_false_pos+of_true_pos)*100))
print(' GLASSFIL SENS/FALSE DISC %3.2f , %3.2f'%(gf_true_pos/real_pos*100, gf_false_pos/(gf_false_pos+gf_true_pos)*100))


juliana_1
juli openface    : Sensitivity/False Disc: 38.37 , 21.74
juli glasses     : Sensitivity/False Disc: 94.19 , 6.72
juli filt glasses: Sensitivity/False Disc: 91.86 , 5.01
beata_1
beat openface    : Sensitivity/False Disc: 47.50 , 54.05
beat glasses     : Sensitivity/False Disc: 90.22 , 58.76
beat filt glasses: Sensitivity/False Disc: 90.22 , 58.76
irmandy_2
irma openface    : Sensitivity/False Disc: 64.48 , 19.32
irma glasses     : Sensitivity/False Disc: 61.00 , 21.59
irma filt glasses: Sensitivity/False Disc: 57.72 , 19.62
david_3b
davi openface    : Sensitivity/False Disc: 75.94 , 37.90
davi glasses     : Sensitivity/False Disc: 85.21 , 19.85
davi filt glasses: Sensitivity/False Disc: 85.21 , 19.85
patrick_3
patr openface    : Sensitivity/False Disc: 68.93 , 44.36
patr glasses     : Sensitivity/False Disc: 81.80 , 16.51
patr filt glasses: Sensitivity/False Disc: 80.33 , 14.65
------------------------------
 OVERALL 
 OPENFACE SENS/FALSE DISC 59.16 , 38.28
 GLASS    SENS/FALS